## Import Modules

In [1]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt


#sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef

plt.style.use('fivethirtyeight')
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set_theme(style='whitegrid', font='Arial', rc={'figure.figsize':(10,5),
            'font.size':14,
            'axes.titlesize':16,
            'axes.labelsize':15,
            'xtick.labelsize': 12,
            'ytick.labelsize': 12,
            'legend.fontsize': 13},color_codes=True)

# CSV
import csv

In [2]:
# room number
a = 'E07'
# number of lags as input (not relevant in this case)
#b = '5'
# number of last timestep to predict
c = 12

In [3]:
# d = number of timestep to predict
for d in range(1, c+1):
    
    if d==0:
        # Timestep as string
        e = 't'
    else:
        # Timestep as string
        e = 't+'+str(d)
    
    
    # Import data
    
    # Read in data and set index

    raw_data = pd.read_csv("\Pre-Processing\data_E07_input_5_output_144.txt", parse_dates=True)
    data = raw_data.copy()
    data['DateTime'] = pd.to_datetime(data['DateTime'])
    data = data.set_index('DateTime')
    
    # Drop columns for Year and Second
    data = data.drop('Second_0', axis = 1)
    
    # Saving data names for later use
    data_list = list(data.columns)
    
    # Drop colums that should not be used as features in this model (columns that refer to the date)
    data = data.drop(data_list[6:86], axis = 1)
    
    # Drop columns of future timestamps that should not be used as input for this model
    if d==0:
        for i in range(1,145):
            v = 't+'+str(i)
            data = data.drop(v, axis = 1)
    else:
        for i in range(d+1,145):
            v = 't+'+str(i)
            data = data.drop(v, axis = 1)

        for i in range(1, d):
            v = 't+'+str(i)
            data = data.drop(v, axis = 1)
    
    
    # parameter that only should be used as input
    if d==0:
        l = ['E07CO2', 'E07Tair', 'E07ElL', 'E07SP', 'E07W', 'E07WT', 't-1', 't-2', 't-3', 't-4','t-5']
    else:
        l = ['E07CO2', 'E07Tair', 'E07ElL', 'E07SP', 'E07W', 'E07WT', 't', 't-1', 't-2', 't-3', 't-4','t-5']
        
    for f in l:
    
    
        # Create data set / dummy variables
    
        # keep only one parameter and the timestep to predict
        to_keep = []
        to_keep.append(f)
        to_keep.append(e)
        
        data_final=data[to_keep]
        data_final.columns.values
        
        # Create X and y
        X = data_final.loc[:, data_final.columns != e]
        y = data_final.loc[:, data_final.columns == e]
        
        # format y to a 1D array
        y = y.values.ravel()
        
        # format X when more than one parameter are used as input
        sc_x = StandardScaler()
        X = sc_x.fit_transform(X)
        X = pd.DataFrame(X)
        
        
        # Logistic Regression Model Fitting
        
        # Split data into training and testing sets (70 % training, 30 % testing)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False, random_state=42)
        
#         # Split testing set into testing and validation sets (15 % training, 15 % validation of the whole data set --> 50/50 % of 30 %)
#         X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
        
        # Print shapes of data sets
        print('Training Data Shape:', X_train.shape)
        print('Training Labels Shape:', y_train.shape)
        print('Testing Data Shape:', X_test.shape)
        print('Testing Labels Shape:', y_test.shape)
#         print('Validation Data Shape:', X_val.shape)
#         print('Validation Labels Shape:', y_val.shape)
        
        # Create and fit model
        logreg = LogisticRegression(max_iter=1000)
        logreg.fit(X_train, y_train)
        
        # accuracy on the training set
        accuracy_train = logreg.score(X_train, y_train)
        accuracy_train_percent = round(accuracy_train*100, 3)

        # Make predictions on the test set
        y_pred = logreg.predict(X_test)
        print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
        
        # accuracy on the test set
        accuracy_test = logreg.score(X_test, y_test)
        accuracy_test_percent = round(accuracy_test*100, 3)
        print(accuracy_test)
        
#         # Make predictions on the validation set
#         y_pred_val = logreg.predict(X_val)
#         print('Accuracy of logistic regression classifier on validation set: {:.2f}'.format(logreg.score(X_val, y_val)))
        
#         # accuracy on the validation set
#         accuracy_val = logreg.score(X_val, y_val)
#         accuracy_val_percent = round(accuracy_val*100, 3)
#         print(accuracy_val)
        
        
        # Analysis of results
        
        # Confusion Matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        print(conf_matrix)
        # check if not only 0s are predicted
        count_no_occ_pred = np.count_nonzero(y_pred == 0)
        print('Number of 0 predicted: ', count_no_occ_pred)
        count_occ_pred = np.count_nonzero(y_pred == 1)
        print('Number of 1 predicted: ', count_occ_pred)
        
        # Classification Report
        print(classification_report(y_test, y_pred))
        
        # Precision
        precision = precision_score(y_test, y_pred)
        precision_percent = round((precision*100),3)
        
        # Recall
        recall = recall_score(y_test, y_pred)
        recall_percent = round((recall*100),3)

        # F1 Score
        f1 = f1_score(y_test, y_pred)
        f1_percent = round((f1*100),3)

        # Matthews Correlation Coefficient
        mcc = matthews_corrcoef(y_test, y_pred)
        
        # ROC
        logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
        fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
        plt.figure()
        plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
        plt.plot([0,1], [0,1], 'r--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic - '+e)
        plt.legend(loc="lower right")
        plt.savefig('Logistic_Regression_ROC_'+a+'_one_parameter_'+f+'_forecast_'+e+'.pdf', bbox_inches='tight', dpi=100)
        plt.close()
        
        
        # Save results
        
        if d==1 and f=='E07CO2':
            # Creating csv file with results for model with all variables
            with open('Logistic_Regression_results_'+a+'_one_parameter.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Predicted Timestep','Input', 'Accuracy - Training Set', 'Accuracy - Test Set',  'Precision', 'Recall',  'F1-Score', 'Matthews Correlation Coefficient'])
                writer.writerow([e, f, accuracy_train, accuracy_test, precision, recall, f1,  mcc])
                
        else:
            # Appending results to existing csv file
            with open('Logistic_Regression_results_'+a+'_one_parameter.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([e, f, accuracy_train, accuracy_test, precision, recall, f1, mcc])

Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.83
0.8334700574241182
[[20191   128]
 [ 3932   129]]
Number of 0 predicted:  24123
Number of 1 predicted:  257
              precision    recall  f1-score   support

         0.0       0.84      0.99      0.91     20319
         1.0       0.50      0.03      0.06      4061

    accuracy                           0.83     24380
   macro avg       0.67      0.51      0.48     24380
weighted avg       0.78      0.83      0.77     24380

Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.83
0.8278917145200985
[[19927   392]
 [ 3804   257]]
Number of 0 predicted:  23731
Number of 1 predicted:  649
              precision    recall  f1-score   support

         0.0       0.84     

C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.83
0.8334290401968827
[[20319     0]
 [ 4061     0]]
Number of 0 predicted:  24380
Number of 1 predicted:  0
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91     20319
         1.0       0.00      0.00      0.00      4061

    accuracy                           0.83     24380
   macro avg       0.42      0.50      0.45     24380
weighted avg       0.69      0.83      0.76     24380



C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.89
0.8894995898277277
[[19875   444]
 [ 2250  1811]]
Number of 0 predicted:  22125
Number of 1 predicted:  2255
              precision    recall  f1-score   support

         0.0       0.90      0.98      0.94     20319
         1.0       0.80      0.45      0.57      4061

    accuracy                           0.89     24380
   macro avg       0.85      0.71      0.75     24380
weighted avg       0.88      0.89      0.88     24380

Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.83
0.8334290401968827
[[20319     0]
 [ 4061     0]]
Number of 0 predicted:  24380
Number of 1 predicted:  0
              precision    recall  f1-score   support

         0.0       0.83      

...

Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.83
0.8329778506972929
[[20308     0]
 [ 4072     0]]
Number of 0 predicted:  24380
Number of 1 predicted:  0
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91     20308
         1.0       0.00      0.00      0.00      4072

    accuracy                           0.83     24380
   macro avg       0.42      0.50      0.45     24380
weighted avg       0.69      0.83      0.76     24380



C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.86
0.8603363412633306
[[19514   794]
 [ 2611  1461]]
Number of 0 predicted:  22125
Number of 1 predicted:  2255
              precision    recall  f1-score   support

         0.0       0.88      0.96      0.92     20308
         1.0       0.65      0.36      0.46      4072

    accuracy                           0.86     24380
   macro avg       0.76      0.66      0.69     24380
weighted avg       0.84      0.86      0.84     24380

Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.83
0.8329778506972929
[[20308     0]
 [ 4072     0]]
Number of 0 predicted:  24380
Number of 1 predicted:  0
              precision    recall  f1-score   support

         0.0       0.83      

C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.87
0.8653814602132895
[[18673  1635]
 [ 1647  2425]]
Number of 0 predicted:  20320
Number of 1 predicted:  4060
              precision    recall  f1-score   support

         0.0       0.92      0.92      0.92     20308
         1.0       0.60      0.60      0.60      4072

    accuracy                           0.87     24380
   macro avg       0.76      0.76      0.76     24380
weighted avg       0.87      0.87      0.87     24380

Training Data Shape: (73137, 1)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 1)
Testing Labels Shape: (24380,)
Accuracy of logistic regression classifier on test set: 0.86
0.8605824446267433
[[18615  1693]
 [ 1706  2366]]
Number of 0 predicted:  20321
Number of 1 predicted:  4059
              precision    recall  f1-score   support

         0.0       0.92   